# Book Recommendation Engine
## Building a Book Recommendation System using Goodbooks-10k dataset
This project creates a recommendation engine that suggests books a user might like based on their past ratings. It uses the K-Nearest Neighbors (KNN) algorithm implemented from scratch, similar to our movie recommendation system.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

### Load the Data
We will use the `books.csv` and `ratings.csv` from the Goodbooks-10k dataset.

In [ ]:
# Load book metadata
books = pd.read_csv('books.csv', usecols=['book_id', 'title'])
books.head()

In [ ]:
# Load user ratings
ratings = pd.read_csv('ratings.csv', usecols=['user_id', 'book_id', 'rating'])
ratings.head()

In [ ]:
# Merge both datasets
book_info = pd.merge(books, ratings, left_on='book_id', right_on='book_id')
book_info.head()

In [ ]:
# Sort by user and book
book_info = book_info.sort_values(['user_id', 'book_id'])
book_info.head()

### Create the User-Item Matrix

In [ ]:
user_item_matrix = book_info.pivot_table(index='user_id', columns='book_id', values='rating')
user_item_matrix.fillna(0, inplace=True)
user_item_matrix.head()

### K-Nearest Neighbors from Scratch
We'll define a function to compute the K-nearest users based on cosine similarity.

In [ ]:
def knn(user_matrix, target_user_id, k=5):
    similarities = cosine_similarity([user_matrix.loc[target_user_id]], user_matrix)[0]
    similarity_scores = list(enumerate(similarities))
    similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)
    nearest_users = [user for user, score in similarity_scores[1:k+1]]
    return nearest_users

In [ ]:
target_user = 1  # Example user
nearest = knn(user_item_matrix, target_user, k=5)
nearest

### Generate Recommendations

In [ ]:
def recommend_books(user_matrix, book_data, target_user_id, nearest_users, num_recommendations=5):
    user_ratings = user_matrix.loc[target_user_id]
    unrated_books = user_ratings[user_ratings == 0].index
    predicted_ratings = {}
    for book_id in unrated_books:
        total = 0
        count = 0
        for user in nearest_users:
            rating = user_matrix.loc[user, book_id]
            if rating != 0:
                total += rating
                count += 1
        if count > 0:
            predicted_ratings[book_id] = total / count
    recommended_books = sorted(predicted_ratings.items(), key=lambda x: x[1], reverse=True)[:num_recommendations]
    return book_data[book_data['book_id'].isin([book[0] for book in recommended_books])]['title']

In [ ]:
recommend_books(user_item_matrix, books, target_user, nearest)